In [1]:
import pandas as pd
import altair as alt
from altair_saver import save

In [3]:
df = pd.read_csv('GLB.Ts_dSST.csv', header=1)
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.29,-0.18,-0.11,-0.19,-0.11,-0.23,-0.21,-0.09,-0.16,-0.23,-0.20,-0.23,-0.19,***,***,-0.14,-0.18,-0.20
1,1881,-0.15,-0.17,0.04,0.04,0.02,-0.20,-0.06,-0.02,-0.13,-0.20,-0.21,-0.10,-0.10,-.11,-.18,0.03,-0.09,-0.18
2,1882,0.15,0.15,0.04,-0.18,-0.16,-0.26,-0.20,-0.05,-0.10,-0.24,-0.16,-0.24,-0.10,-.09,.06,-0.10,-0.17,-0.17
3,1883,-0.31,-0.39,-0.13,-0.17,-0.20,-0.12,-0.08,-0.15,-0.20,-0.14,-0.22,-0.16,-0.19,-.20,-.31,-0.16,-0.12,-0.19
4,1884,-0.15,-0.08,-0.37,-0.42,-0.36,-0.40,-0.34,-0.26,-0.27,-0.24,-0.30,-0.28,-0.29,-.28,-.13,-0.39,-0.34,-0.27


In [4]:
df.drop(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'D-N', 'DJF', 'MAM', 'JJA', 'SON'], axis = 1, inplace = True)
df.head()

,Year,J-D
0,1880,-0.19
1,1881,-0.10
2,1882,-0.10
3,1883,-0.19
4,1884,-0.29


In [5]:
df.dtypes
df.head(20)

,Year,J-D
0,1880,-0.19
1,1881,-0.10
2,1882,-0.10
3,1883,-0.19
4,1884,-0.29
5,1885,-0.31
6,1886,-0.32
7,1887,-0.35
8,1888,-0.18
9,1889,-0.11


In [6]:
av_80_99 = 0
vals = df['J-D']
vals_80_99 = vals[0:20]
av_80_99 = vals_80_99.sum()/len(vals_80_99)


av_80_99

-0.22849999999999998

In [7]:
labels = df[-4:]
min_val = df['J-D'].argmin()
labels = labels.append(df.loc[min_val])
max1950_val = df[0:71]['J-D'].argmax()
max2000_val = df[70:121]['J-D'].argmax()
labels = labels.append(df.loc[max1950_val])
labels = labels.append(df.loc[max2000_val+70])
labels

,Year,J-D
134,2014.0,0.73
135,2015.0,0.86
136,2016.0,0.99
137,2017.0,0.90
24,1904.0,-0.49
64,1944.0,0.21
118,1998.0,0.62


In [8]:
year_labels = []

for index, row in labels.iterrows():
    fweight = 200
    if int(row["Year"]) == 2016:
        fweight = 'bold'
    x_shift = 10
    if int(row["Year"]) == 1998 or int(row["Year"]) == 1904:
        x_shift = -40
    
    text = alt.Chart({'values':[{'Year': row["Year"], 'J-D': row["J-D"]}]}).mark_text(
        text=str(int(row["Year"])), fontSize = 15, fontWeight = fweight,     align='left',
        baseline='middle',
        dx= x_shift
    ).encode(
        x='Year:Q', y='J-D:Q'
    )
    year_labels.append(text)


In [9]:
line = alt.Chart(pd.DataFrame({'y': [av_80_99]})).mark_rule().encode(y='y')

text1 = alt.Chart({'values':[{'x': 2009, 'y': -0.10}]}).mark_text(
    text=['HOTTER THAN THE', '1880-1899 AVERAGE'], fontSize = 15, fontWeight = 200
).encode(
    x='x:Q', y='y:Q'
)

text2 = alt.Chart({'values':[{'x': 2015, 'y': -0.29}]}).mark_text(
    text='COLDER', fontSize = 15, fontWeight = 200
).encode(
    x='x:Q', y='y:Q'
)

chart = alt.Chart(df, title=['Annual Global Surface Temperature,', 'Relative to Late 19th Century Average']).mark_point(filled = False, fill="#F26C08", size = 80,color="#000000", opacity = 1).encode(
    x = alt.X(field='Year', 
              title = '',
              type = 'quantitative',
              scale = alt.Scale(
                  zero = False
              ),
             axis = alt.Axis(
                 tickCount=14,
                 format = "1000f",
                 tickMinStep = 10,
                 labelPadding=15
             )),
    y = alt.Y(field='J-D', 
              title = '',
              type = 'quantitative', 
              scale = alt.Scale(
                  zero = False
              ),
             axis = alt.Axis(tickCount=9,
             labelExpr = "format(datum.value, '+.1f') +'°'",
             tickMinStep = 0.2,
             labelPadding=15
             ))
)

final_chart = alt.layer(chart, text1, text2, line, *(year_labels[i] for i in range(len(year_labels)))).properties(width = 950, 
              height = 500, 
              background = '#FFFFFF', 
              padding = 25
             ).configure_view(strokeWidth = 0
             ).configure_axis(ticks = False,
                              domain = False,
                              labelColor = "#999999",
                              gridDash = [2, 2],
                              gridWidth = 0.7,
                              gridDashOffset = 1000,
                              titleFont = 'Helvetica',
                              labelFont = 'Helvetica',
                              titleFontSize = 14,
                              labelFontSize = 14
             ).configure_text(font = 'Helvetica'
             ).configure_title(
                                align='left',
                                anchor='start', 
                                dx = 60,
                                offset=-40,
                                fontSize = 18,
                                fontWeight = 'bold'
                            )

final_chart.save("chart.html")
# final_chart